Utilizei Python 3 e as bibliotecas pandas, NumPy e Matplotlib.  Os códigos rodam no Jupyter Notebook. 

In [1]:
import pandas as pd
import plotly.express as px

Aqui importo a biblioteca pandas e a nomeio como 'pd' para facilitar a digitação do código.

In [2]:
xls = pd.ExcelFile('SIC 584922017103.xlsx')

Como é um arquivo Excel, uso a classe ExcelFile e chamo o dataframe de "xls".

In [3]:
xls.sheet_names

['Base de Dados', 'Metodologia', 'Pedido', 'Campos da Tabela']

Verifico as tabelas que existem dentro do arquivo. Existem 4 e as que me interessaram foi "Base de Dados" e "Campos da Tabela".

In [4]:
xls.parse('Base de Dados')

,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,NOME_DELEGACIA,CIDADE,ANO_BO,NUM_BO,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,NOME_DELEGACIA_CIRC,...,DESCRICAO_DEFICIENCIA,DESCR_RELACIONAMENTO,FLAG_VITIMA_FATAL,SEXO_PESSOA,IDADE_PESSOA,COR_CUTIS,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO,NACIONALIDADE_PESSOA,NATURALIDADE_PESSOA
0,100217,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,ALTAIR,2019,4,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,...,NaN,NaN,NaN,F,43.0,Parda,PRENDAS DOMESTICAS,1 Grau completo,BRASILEIRA,ORLANDIA -SP
1,100217,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,ALTAIR,2019,4,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,...,NaN,NaN,NaN,F,19.0,Parda,ESTUDANTE,1 Grau completo,BRASILEIRA,NOVA GRANADA -SP
2,100217,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,ALTAIR,2019,4,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,...,NaN,NaN,NaN,F,43.0,Parda,PRENDAS DOMESTICAS,1 Grau completo,BRASILEIRA,ORLANDIA -SP
3,100217,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,ALTAIR,2019,4,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,...,NaN,NaN,NaN,F,19.0,Parda,ESTUDANTE,1 Grau completo,BRASILEIRA,NOVA GRANADA -SP
4,100217,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,ALTAIR,2019,12,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,...,NaN,NaN,NaN,F,43.0,Branca,PRENDAS DOMESTICAS,1 Grau incompleto,BRASILEIRA,NOSSA SENHORA GLORIA -SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57773,100912,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.PLANTÃO BEBEDOURO,VISTA ALEGRE DO ALTO,2019,391,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.VISTA ALEGRE DO ALTO,...,NaN,UNIAO ESTAVEL,N,F,27.0,Branca,LAVRADOR(A),1 Grau completo,BRASILEIRA,MORRO DO CHAPÉU/BA
57774,100912,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.PLANTÃO BEBEDOURO,VISTA ALEGRE DO ALTO,2019,391,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.VISTA ALEGRE DO ALTO,...,NaN,NENHUMA RELACAO,N,F,28.0,Parda,LAVRADOR(A),1 Grau incompleto,BRASILEIRA,BARRINHA -SP
57775,100912,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.PLANTÃO BEBEDOURO,VISTA ALEGRE DO ALTO,2019,391,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.VISTA ALEGRE DO ALTO,...,NaN,UNIAO ESTAVEL,N,F,27.0,Branca,LAVRADOR(A),1 Grau completo,BRASILEIRA,MORRO DO CHAPÉU/BA
57776,100912,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.PLANTÃO BEBEDOURO,VISTA ALEGRE DO ALTO,2019,620,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.VISTA ALEGRE DO ALTO,...,NaN,NaN,NaN,F,25.0,Branca,FISIOTERAPEUTA,Superior completo,BRASILEIRA,PIRANGI -SP


Comando para checar o que compõe a tabela "Base de Dados". É realmente o que eu preciso, mas são muitas colunas (53), então é necessário uma limpeza.

In [5]:
xls.parse('Campos da Tabela')

,A PRESENTE TABELA TEM POR FINALIDADE ESCLARECER OS CAMPOS CONTIDOS NA BASE DE DADOS,Unnamed: 1
0,*Resultado Null indica que o campo se encontra...,NaN
1,NaN,NaN
2,Campos,Descrição
3,NOME_DEPARTAMENTO,Departamento responsável pelo registro
4,NOME_SECCIONAL,Delegacia Seccional responsável pelo registro
5,NOME_DELEGACIA,Delegacia responsável pelo registro
6,CIDADE,Cidade de Registro
7,ANO_BO,Ano do BO
8,NUM_BO,Número do BO
9,NOME_DEPARTAMENTO_CIRC,Departamento de Circunscrição


A tabela "Campos da Tabela" mostra o que significa cada coluna, então ficará mais fácil para entender cada uma e filtrar o que é realmente necessário para a história que quero contar.

In [6]:
campos = pd.read_excel(xls, 'Campos da Tabela')

Criei um novo dataframe chamado "campos" somente com a tabela "Campos da Tabela", pois há muito espaço e dificultou a leitura de algumas linhas, então irei limpar.

In [7]:
campos.columns

Index(['A PRESENTE TABELA TEM POR FINALIDADE ESCLARECER OS CAMPOS CONTIDOS NA BASE DE DADOS', 'Unnamed: 1'], dtype='object')

Confirmei o nome das colunas. Como não foi especificado na tabela, foi usado a primeira linha como nome de cada coluna. 

In [8]:
campos = campos.rename(columns={'A PRESENTE TABELA TEM POR FINALIDADE ESCLARECER OS CAMPOS CONTIDOS NA BASE DE DADOS':'CAMPO'})
campos = campos.rename(columns={'Unnamed: 1':'DESCRIÇÃO'})
campos = campos.drop([0, 1, 2])

In [9]:
campos

,CAMPO,DESCRIÇÃO
3,NOME_DEPARTAMENTO,Departamento responsável pelo registro
4,NOME_SECCIONAL,Delegacia Seccional responsável pelo registro
5,NOME_DELEGACIA,Delegacia responsável pelo registro
6,CIDADE,Cidade de Registro
7,ANO_BO,Ano do BO
8,NUM_BO,Número do BO
9,NOME_DEPARTAMENTO_CIRC,Departamento de Circunscrição
10,NOME_SECCIONAL_CIRC,Seccional de Circunscrição
11,NOME_DELEGACIA_CIRC,Delegacia de Circunscrição
12,NOME_MUNICIPIO_CIRC,Município da Delegacia de Circunscrição


 Alterei o nome das colunas para algo mais amigável "CAMPO" e "DESCRIÇÃO" e retirei as três primeiras linhas da tabela.

In [10]:
df = pd.read_excel(xls, 'Base de Dados')

Criei um novo dataframe chamado "df" somente com a tabela "Base de Dados".

In [11]:
df.sample(5)

,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,NOME_DELEGACIA,CIDADE,ANO_BO,NUM_BO,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,NOME_DELEGACIA_CIRC,...,DESCRICAO_DEFICIENCIA,DESCR_RELACIONAMENTO,FLAG_VITIMA_FATAL,SEXO_PESSOA,IDADE_PESSOA,COR_CUTIS,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO,NACIONALIDADE_PESSOA,NATURALIDADE_PESSOA
29568,100310,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.FRANCA,DEL.POL.MIGUELOPOLIS,MIGUELOPOLIS,2019,893,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.FRANCA,DEL.POL.MIGUELOPOLIS,...,NaN,NaN,NaN,F,61.0,Parda,PRENDAS DOMESTICAS,1 Grau completo,BRASILEIRA,LUZ -MG
32610,100229,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DDM OLIMPIA,OLIMPIA,2020,40,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,01º D.P. OLIMPIA,...,NaN,NaN,NaN,F,26.0,Parda,NAO INFORMADA,NaN,BRASILEIRA,AFONSO CUNHA -MA
8940,100323,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.FRANCA,DEL.POL.BATATAIS,BATATAIS,2019,161,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.FRANCA,DEL.POL.BATATAIS,...,NaN,NaN,NaN,F,49.0,Branca,NaN,1 Grau completo,BRASILEIRA,BATATAIS -SP
42229,100919,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.RIBEIRAO PRETO,CENTRAL POL.JUD-RIB. PRETO,RIBEIRAO PRETO,2019,4034,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.RIBEIRAO PRETO,02º D.P. RIBEIRAO PRETO,...,NaN,NaN,NaN,F,13.0,Parda,ESTUDANTE,NaN,BRASILEIRA,RIBEIRAO PRETO -SP
45122,100919,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.RIBEIRAO PRETO,CENTRAL POL.JUD-RIB. PRETO,RIBEIRAO PRETO,2020,1910,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.RIBEIRAO PRETO,06º D.P. RIBEIRAO PRETO,...,NaN,PARENTESCO,N,F,39.0,Parda,AUTONOMO(A),NaN,BRASILEIRA,RIBEIRAO PRETO -SP


Mesmo com a descrição de cada campo, algumas coisas eu ainda não consegui entender, então peguei 5 exemplos aleatórios para analisar o que cada um significa e selecionar melhor as colunas que desejo manter.

In [12]:
df = df[['CIDADE', 'DATA_COMUNICACAO_BO', 'FLAG_STATUS', 'VIOLENCIA_DOMESTICA', 'RUBRICA', 'DESCR_SUBTIPOLOCAL', 'DESCR_RELACIONAMENTO', 'IDADE_PESSOA', 'DESCR_PROFISSAO', 'DESCR_GRAU_INSTRUCAO']]

In [13]:
df.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
49300,S.CARLOS,2020-04-15,Consumado,VIOLÊNCIA DOMESTICA,Dano (art. 163),CASA,UNIAO ESTAVEL,22.0,CABELEIREIRO(A),2 Grau completo
19473,FRANCA,2020-06-28,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art 129 § 9º),CASA,UNIAO ESTAVEL,36.0,SAPATEIRO(A),NaN
46416,S.PAULO,2020-05-21,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),CASA,NaN,51.0,AUXILIAR ADMINISTRATIVO,NaN
15375,FERNANDO PRESTES,2020-03-13,Consumado,NaN,Ameaça (art. 147),VIA PÚBLICA,NaN,32.0,AJUDANTE,1 Grau completo
15466,FRANCA,2019-01-03,Consumado,NaN,Ameaça (art. 147),CASA,NaN,44.0,NaN,NaN


Apenas com as colunas que desejo, fica mais fácil a análise dos dados. A forma como a idade está apresentada me incomoda um pouco, então vou corrigir.

In [14]:
df.dtypes

CIDADE                          object
DATA_COMUNICACAO_BO     datetime64[ns]
FLAG_STATUS                     object
VIOLENCIA_DOMESTICA             object
RUBRICA                         object
DESCR_SUBTIPOLOCAL              object
DESCR_RELACIONAMENTO            object
IDADE_PESSOA                   float64
DESCR_PROFISSAO                 object
DESCR_GRAU_INSTRUCAO            object
dtype: object

O tipo da coluna está como float, mas vou mudar para inteiro, já que ninguem diz "minha idade é 51,3".

In [15]:
import numpy as np

In [16]:
df['IDADE_PESSOA'] = df['IDADE_PESSOA'].fillna(0).astype(np.int64)

Importei a biblioteca numpy como "np" e fiz a mudança do tipo da coluna, colocando "0" nas que não apresentavam valores.

In [17]:
df.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
16471,FRANCA,2019-07-17,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art. 129),VIA PÚBLICA,ENVOLVIMENTO AMOROSO,36,PENSIONISTA,1 Grau incompleto
27626,JABOTICABAL,2020-05-27,Consumado,NaN,Injúria (art. 140),OUTROS,NaN,23,PROMOTOR DE VENDAS,NaN
26504,JABOTICABAL,2019-02-12,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,UNIAO ESTAVEL,29,AUTONOMO(A),1 Grau completo
51757,S.RITA PASSA QUATRO,2020-03-17,Consumado,NaN,Lesão corporal (art. 129),CASA,NaN,15,ESTUDANTE,2 Grau incompleto
122,ALTINOPOLIS,2019-03-21,Consumado,NaN,Difamação (art. 139),CASA,NaN,49,TAXISTA,2 Grau completo


In [18]:
df.to_csv('violencia-domestica-rp.csv', encoding='utf8', index=False)

É isso mesmo que quero, então aproveitei pra exportar a tabela caso eu queira utilizar novamente uma outra hora.

Como eu desejo falar especificamente do período da quarentena, preciso separar as datas por mês. A coluna "DATA_COMUNICACAO_BO" já está com o tipo correto, como vi anteriormente com o comando dtype, logo vai ser simples.

In [19]:
df2019 = df[df['DATA_COMUNICACAO_BO'].dt.year == 2019]

In [20]:
df2019.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
37148,RIBEIRAO PRETO,2019-01-18,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,UNIAO ESTAVEL,48,PRENDAS DOMESTICAS,NaN
56310,TAQUARITINGA,2019-11-14,Consumado,NaN,Ameaça (art. 147),CASA,NaN,16,ESTUDANTE,NaN
12326,BRODOWSKI,2019-09-04,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art. 129),VIA PÚBLICA,ENVOLVIMENTO AMOROSO,18,ESTUDANTE,2 Grau incompleto
19141,FRANCA,2019-12-22,Consumado,NaN,Ameaça (art. 147),VIA PÚBLICA,NaN,23,SAPATEIRO(A),2 Grau incompleto
12150,BRODOWSKI,2019-01-02,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),CASA,NENHUMA RELACAO,35,AUXILIAR DE SERVIÇOS GERAIS,1 Grau completo


Primeiro, separei por ano. Irei fazer o mesmo com 2020. Perceba que criei um novo dataframe chamado "df2019" para não sobreescrever as informações do outro dataframe, criando também um novo dataframe para 2020, chamado "df2020"

In [21]:
df2020 = df[df['DATA_COMUNICACAO_BO'].dt.year == 2020]

In [22]:
df2020.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
17657,FRANCA,2020-04-14,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,CASAMENTO,40,NaN,NaN
5839,ARARAQUARA,2020-06-14,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),CASA,CASAMENTO,54,PRENDAS DOMESTICAS,Superior incompleto
54442,SERTAOZINHO,2020-04-08,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art. 129),CASA,UNIAO ESTAVEL,30,NAO INFORMADA,NaN
41172,RIBEIRAO PRETO,2020-03-04,Consumado,NaN,Ameaça (art. 147),APARTAMENTO,NaN,65,APOSENTADO(A),NaN
49010,S.CARLOS,2020-01-06,Consumado,NaN,Ameaça (art. 147),CASA,NaN,30,TECNICO EM ENFERMAGEM,NaN


In [23]:
df2019 = df2019.sort_values(by='DATA_COMUNICACAO_BO')
df2020 = df2020.sort_values(by='DATA_COMUNICACAO_BO')

In [24]:
df2019.tail(3)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
5526,ARARAQUARA,2019-12-31,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,PARENTESCO,56,PRENDAS DOMESTICAS,2 Grau completo
8107,BARRETOS,2019-12-31,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),APARTAMENTO,ENVOLVIMENTO AMOROSO,36,LAVRADOR(A),1 Grau incompleto
5530,ARARAQUARA,2019-12-31,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,PARENTESCO,79,APOSENTADO(A),Analfabeto


In [25]:
df2020.tail(3)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
41805,RIBEIRAO PRETO,2020-06-30,Consumado,NaN,Injúria (art. 140),APARTAMENTO,NaN,67,NaN,NaN
27304,JABOTICABAL,2020-06-30,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),AREA COMUM-,UNIAO ESTAVEL,33,PRENDAS DOMESTICAS,2 Grau completo
27305,JABOTICABAL,2020-06-30,Consumado,NaN,Estupro de vulneravel (art.217-A),CASA,NaN,7,NaN,NaN


Primeiro, eu ordenei por data, do mês 1 ao 12. Depois, eu vi o final de cada uma para ver até qual mês iria. Como vai até o mês de julho na na tabela de 2020, farei o mesmo filtro na de 2019.

In [26]:
dfquar19 = df2019[(df2019['DATA_COMUNICACAO_BO'] >= '2019-03-01') & (df2019['DATA_COMUNICACAO_BO'] <= '2019-04-01')]

In [27]:
dfquar20 = df2020[(df2020['DATA_COMUNICACAO_BO'] >= '2020-03-01') & (df2020['DATA_COMUNICACAO_BO'] <= '2020-04-01')]

In [28]:
dfquar19.head(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
28677,MATAO,2019-03-01,Consumado,NaN,Dano (art. 163),CASA,NaN,59,NAO INFORMADA,Superior completo
1286,ARARAQUARA,2019-03-01,Consumado,NaN,Ameaça (art. 147),CASA,NaN,75,COSTUREIRO(A),1 Grau incompleto
52303,S.SIMAO,2019-03-01,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),VIA PÚBLICA,ENVOLVIMENTO AMOROSO,31,NaN,NaN
33170,ORLANDIA,2019-03-01,Consumado,NaN,Lesão corporal (art. 129),POSTO DE GASOLINA,NaN,24,DESEMPREGADO(A),2 Grau incompleto
1285,ARARAQUARA,2019-03-01,Consumado,NaN,Injúria (art. 140),PREFEITURA MUNICIPAL,NaN,29,FUNCION.PUBLICO MUNICIPAL,Superior completo


In [29]:
dfquar20.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
49976,S.CARLOS,2020-03-17,Consumado,NaN,Ameaça (art. 147),CASA,NaN,59,FUNCION.PUBLICO MUNICIPAL,Superior completo
12062,BORBOREMA,2020-03-16,Consumado,NaN,Lesão corporal (art. 129),VIA PÚBLICA,NaN,25,NaN,1 Grau completo
49200,S.CARLOS,2020-03-05,Consumado,NaN,Ameaça (art. 147),CASA,NaN,21,ESTUDANTE,NaN
36101,PORTO FERREIRA,2020-03-16,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art. 129),VIA PÚBLICA,ENVOLVIMENTO AMOROSO,24,ESTUDANTE,NaN
56926,TAQUARITINGA,2020-03-19,Consumado,NaN,Ameaça (art. 147),CASA,NaN,17,NaN,2 Grau incompleto


Agora que tenho somente os meses que preciso, posso começar com minha matéria, fazendo a leitura dos dados nos dataframes específicos.

In [30]:
dfquar19.to_csv('quar19.csv', index=False, encoding ='utf8')
dfquar20.to_csv('quar20.csv', index=False, encoding ='utf8')

In [31]:
dfquar19.shape

(3704, 10)

In [32]:
dfquar20.shape

(2979, 10)

Usei o comando shape para ver a quantidade de linhas em cada tabela, estimando, assim, o total de casos de cada ano.

In [33]:
dfquar19['CIDADE'].unique()

array(['MATAO                                   ',
       'ARARAQUARA                              ',
       'S.SIMAO                                 ',
       'ORLANDIA                                ',
       'LUIS ANTONIO                            ',
       'MIGUELOPOLIS                            ',
       'PORTO FERREIRA                          ',
       'RIFAINA                                 ',
       'FRANCA                                  ',
       'CANDIDO RODRIGUES                       ',
       'GUAIRA                                  ',
       'CRAVINHOS                               ',
       'BATATAIS                                ',
       'BARRETOS                                ',
       'JARDINOPOLIS                            ',
       'COLOMBIA                                ',
       'S.ROSA DE VITERBO                       ',
       'MORRO AGUDO                             ',
       'IBITINGA                                ',
       'ITUVERAVA              

Preciso filtrar por cidade, então tenho que ter certeza que os nomes estão escritos corretamente. Há um grande espaço neles, portanto irei remove-los.

In [34]:
dfquar19['CIDADE'] = dfquar19['CIDADE'].map(str.strip)

<ipython-input-34-d0050541a4c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfquar19['CIDADE'] = dfquar19['CIDADE'].map(str.strip)


In [35]:
dfquar19['CIDADE'].unique()

array(['MATAO', 'ARARAQUARA', 'S.SIMAO', 'ORLANDIA', 'LUIS ANTONIO',
       'MIGUELOPOLIS', 'PORTO FERREIRA', 'RIFAINA', 'FRANCA',
       'CANDIDO RODRIGUES', 'GUAIRA', 'CRAVINHOS', 'BATATAIS', 'BARRETOS',
       'JARDINOPOLIS', 'COLOMBIA', 'S.ROSA DE VITERBO', 'MORRO AGUDO',
       'IBITINGA', 'ITUVERAVA', 'GUARA', 'BORBOREMA', 'OLIMPIA', 'ALTAIR',
       'PATROCINIO PAULISTA', 'RIBEIRAO PRETO', 'JABOTICABAL', 'S.CARLOS',
       'SERTAOZINHO', 'BARRINHA', 'ITAPOLIS', 'BEBEDOURO',
       'FERNANDO PRESTES', 'PONTAL', 'PEDREGULHO', 'BRODOWSKI', 'TAIACU',
       'IGARAPAVA', 'DOURADO', 'S.JOAQUIM DA BARRA', 'SERRANA',
       'MONTE ALTO', 'AMERICO BRASILIENSE', 'TAQUARITINGA', 'GUATAPARA',
       'PITANGUEIRAS', 'VIRADOURO', 'PIRANGI', 'MONTE AZUL PAULISTA',
       'TAIUVA', 'ALTINOPOLIS', 'S.LUCIA', 'DESCALVADO', 'DUMONT',
       'S.RITA PASSA QUATRO', 'CAJURU', 'GUARACI', 'GUARIBA',
       'SERRA AZUL', 'IBATE', 'IPUA', 'RIBEIRAO BONITO', 'NOVA EUROPA',
       'NUPORANGA', 'SALES OLIVE

Foi me dado um aviso, mas funcionou. Irei ignorar. Agora, filtrar.

In [36]:
dfquar19rp = dfquar19[dfquar19['CIDADE']=='RIBEIRAO PRETO']

In [37]:
dfquar19rp.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
42513,RIBEIRAO PRETO,2019-03-26,Consumado,NaN,Ameaça (art. 147),CASA,NaN,26,AUXILIAR DE COZINHA,1 Grau completo
37782,RIBEIRAO PRETO,2019-03-22,Consumado,NaN,Injúria (art. 140),APARTAMENTOS,NaN,36,DESEMPREGADO(A),2 Grau completo
37639,RIBEIRAO PRETO,2019-03-12,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),APARTAMENTO,CASAMENTO,45,TECNICO EM ENFERMAGEM,2 Grau completo
37806,RIBEIRAO PRETO,2019-03-26,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),CASA,ENVOLVIMENTO AMOROSO,53,MONITOR(A),NaN
42529,RIBEIRAO PRETO,2019-03-29,Consumado,NaN,Ameaça (art. 147),CASA,NaN,35,OUTROS,NaN


Na coluna "CIDADE", peço apenas as linhas que contenham "RIBEIRAO PRETO". Agora tenho somente os crimes cometidos em Ribeirão Preto. Irei fazer mais algumas filtragens, com o mesmo comando usado anteriormente, e também no dataframe dfquar20. Achei melhor separar os dataframes, colocando um "rp" na frente.

In [38]:
dfquar19rp.shape

(645, 10)

In [39]:
dfquar20['CIDADE'].unique()

array(['RIBEIRAO PRETO                          ',
       'MONTE AZUL PAULISTA                     ',
       'GUARIBA                                 ',
       'ITUVERAVA                               ',
       'BARRETOS                                ',
       'S.JOAQUIM DA BARRA                      ',
       'JARDINOPOLIS                            ',
       'SERTAOZINHO                             ',
       'CRAVINHOS                               ',
       'FRANCA                                  ',
       'IBITINGA                                ',
       'S.CARLOS                                ',
       'TAQUARITINGA                            ',
       'BATATAIS                                ',
       'ARARAQUARA                              ',
       'SERRANA                                 ',
       'PORTO FERREIRA                          ',
       'IGARAPAVA                               ',
       'RIBEIRAO BONITO                         ',
       'CAJOBI                 

In [40]:
dfquar20['CIDADE'] = dfquar20['CIDADE'].map(str.strip)

<ipython-input-40-f37ce8656960>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfquar20['CIDADE'] = dfquar20['CIDADE'].map(str.strip)


In [41]:
dfquar20['CIDADE'].unique()

array(['RIBEIRAO PRETO', 'MONTE AZUL PAULISTA', 'GUARIBA', 'ITUVERAVA',
       'BARRETOS', 'S.JOAQUIM DA BARRA', 'JARDINOPOLIS', 'SERTAOZINHO',
       'CRAVINHOS', 'FRANCA', 'IBITINGA', 'S.CARLOS', 'TAQUARITINGA',
       'BATATAIS', 'ARARAQUARA', 'SERRANA', 'PORTO FERREIRA', 'IGARAPAVA',
       'RIBEIRAO BONITO', 'CAJOBI', 'JABOTICABAL', 'BEBEDOURO',
       'DESCALVADO', 'BOA ESPERANCA DO SUL', 'ITAPOLIS', 'COLOMBIA',
       'S.SIMAO', 'S.RITA PASSA QUATRO', 'PIRANGI', 'COLINA',
       'PATROCINIO PAULISTA', 'CRISTAIS PAULISTA', 'RINCAO',
       'VISTA ALEGRE DO ALTO', 'S.ANTONIO DA ALEGRIA', 'CAMPINAS',
       'CAJURU', 'IBATE', 'NUPORANGA', 'SEVERINIA', 'IPUA',
       'PITANGUEIRAS', 'FERNANDO PRESTES', 'MONTE ALTO', 'MATAO',
       'GUAIRA', 'GUARA', 'SERRA AZUL', 'LUIS ANTONIO', 'BARRINHA',
       'AMERICO BRASILIENSE', 'MIGUELOPOLIS', 'TABATINGA', 'JERIQUARA',
       'GUARACI', 'JABORANDI', 'PONTAL', 'S.JOSE DA BELA VISTA',
       'GUATAPARA', 'OLIMPIA', 'S.ROSA DE VITERBO', 'MORR

In [42]:
dfquar20rp = dfquar20[dfquar20['CIDADE']=='RIBEIRAO PRETO']

In [43]:
dfquar20rp.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
41188,RIBEIRAO PRETO,2020-03-05,Consumado,NaN,Ameaça (art. 147),CASA,CONHECIDO,45,OPERADOR DE CAIXA,NaN
45614,RIBEIRAO PRETO,2020-03-14,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art 129 § 9º),VIA PÚBLICA,PARENTESCO,16,ESTUDANTE,2 Grau incompleto
45648,RIBEIRAO PRETO,2020-03-18,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,UNIAO ESTAVEL,23,AUTONOMO(A),2 Grau completo
41363,RIBEIRAO PRETO,2020-03-23,Consumado,NaN,Injúria (art. 140),OUTROS,NaN,42,MEDICO(A),Superior completo
41205,RIBEIRAO PRETO,2020-03-06,Consumado,NaN,Ameaça (art. 147),CASA,PARENTESCO,36,MASSOTERAPEUTA,Superior completo


In [44]:
dfquar20rp.shape

(533, 10)

In [45]:
dfquar19rp['VIOLENCIA_DOMESTICA'].unique()

array([nan, 'VIOLÊNCIA DOMESTICA'], dtype=object)

In [46]:
dfquar19rpvd = dfquar19rp[dfquar19rp['VIOLENCIA_DOMESTICA']=='VIOLÊNCIA DOMESTICA']

In [47]:
dfquar19rpvd.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
42457,RIBEIRAO PRETO,2019-03-20,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art. 129),VIA PÚBLICA,ENVOLVIMENTO AMOROSO,36,VENDEDOR(A),NaN
42459,RIBEIRAO PRETO,2019-03-20,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),VIA PÚBLICA,ENVOLVIMENTO AMOROSO,36,VENDEDOR(A),NaN
37819,RIBEIRAO PRETO,2019-03-27,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art 129 § 9º),APARTAMENTO,PARENTESCO,60,APOSENTADO(A),2 Grau completo
37628,RIBEIRAO PRETO,2019-03-11,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),VIA PÚBLICA,CASAMENTO,46,MANICURE,NaN
37760,RIBEIRAO PRETO,2019-03-20,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),CASA,NENHUMA RELACAO,27,PRENDAS DOMESTICAS,NaN


In [48]:
dfquar19rpvd.shape

(290, 10)

In [49]:
dfquar20rp['VIOLENCIA_DOMESTICA'].unique()

array([nan, 'VIOLÊNCIA DOMESTICA'], dtype=object)

In [50]:
dfquar20rpvd = dfquar20rp[dfquar20rp['VIOLENCIA_DOMESTICA']=='VIOLÊNCIA DOMESTICA']

In [51]:
dfquar20rpvd.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
41293,RIBEIRAO PRETO,2020-03-16,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,ENVOLVIMENTO AMOROSO,0,NaN,NaN
41151,RIBEIRAO PRETO,2020-03-02,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),APARTAMENTO,CASAMENTO,40,COZINHEIRO(A),NaN
41167,RIBEIRAO PRETO,2020-03-04,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,UNIAO ESTAVEL,34,DESEMPREGADO(A),NaN
41296,RIBEIRAO PRETO,2020-03-16,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,CASAMENTO,31,PRENDAS DOMESTICAS,NaN
41179,RIBEIRAO PRETO,2020-03-05,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,CASAMENTO,38,AUXILIAR DE LIMPEZA,NaN


In [52]:
dfquar20rpvd.shape

(297, 10)

Filtrei também pelos crimes que se enquadravam como violência doméstica e criei um novo dataframe para cada nomeados "dfquar19rpvd" e "dfquar20rpvd". Faço assim para não sobreescrever os outros dataframes, caso precise de alguma informação que esteja neles.

In [53]:
dfquar19rpvd['DESCR_RELACIONAMENTO'].unique()

array(['CASAMENTO                               ',
       'CONHECIDO                               ',
       'UNIAO ESTAVEL                           ',
       'ENVOLVIMENTO AMOROSO                    ',
       'PARENTESCO                              ',
       'VIZINHANCA                              ',
       'NENHUMA RELACAO                         '], dtype=object)

In [54]:
dfquar19rpvd['DESCR_RELACIONAMENTO'] = dfquar19rpvd['DESCR_RELACIONAMENTO'].map(str.strip)

<ipython-input-54-e94ab5bfc7a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfquar19rpvd['DESCR_RELACIONAMENTO'] = dfquar19rpvd['DESCR_RELACIONAMENTO'].map(str.strip)


In [55]:
dfquar19rpvd['DESCR_RELACIONAMENTO'].unique()

array(['CASAMENTO', 'CONHECIDO', 'UNIAO ESTAVEL', 'ENVOLVIMENTO AMOROSO',
       'PARENTESCO', 'VIZINHANCA', 'NENHUMA RELACAO'], dtype=object)

In [56]:
dfquar19rpvd = dfquar19rpvd.set_index("DESCR_RELACIONAMENTO")

In [57]:
dfquar19rpvd = dfquar19rpvd.drop("CONHECIDO", axis=0)
dfquar19rpvd = dfquar19rpvd.drop("PARENTESCO", axis=0)
dfquar19rpvd = dfquar19rpvd.drop("VIZINHANCA", axis=0)
dfquar19rpvd = dfquar19rpvd.drop("NENHUMA RELACAO", axis=0)

In [58]:
dfquar19rpvd.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
DESCR_RELACIONAMENTO,,,,,,,,,
UNIAO ESTAVEL,RIBEIRAO PRETO,2019-03-03,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,24,AUTONOMO(A),2 Grau completo
ENVOLVIMENTO AMOROSO,RIBEIRAO PRETO,2019-03-23,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art. 129),CASA,20,BABÁ,1 Grau incompleto
UNIAO ESTAVEL,RIBEIRAO PRETO,2019-03-18,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),CASA,29,FAXINEIRO(A),NaN
CASAMENTO,RIBEIRAO PRETO,2019-03-12,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),APARTAMENTO,45,TECNICO EM ENFERMAGEM,2 Grau completo
UNIAO ESTAVEL,RIBEIRAO PRETO,2019-04-01,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),CASA,46,AUXILIAR DE LIMPEZA,NaN


In [59]:
dfquar19rpvd.shape

(239, 9)

Com estes comandos, eu transformei a coluna "DESCR_RELACIONAMENTO" em índice para facilitar a filtragem por linhas, selecionando apenas os registros onde a vítima tinha algum relacionamento amoroso com o agressor. Farei o mesmo com o dataframe "dfquar20rpvd"

In [60]:
dfquar20rpvd['DESCR_RELACIONAMENTO'].unique()

array(['CASAMENTO                               ',
       'UNIAO ESTAVEL                           ',
       'ENVOLVIMENTO AMOROSO                    ',
       'PARENTESCO                              ',
       'CONHECIDO                               ',
       'NENHUMA RELACAO                         ', nan], dtype=object)

In [61]:
dfquar20rpvd = dfquar20rpvd.dropna(subset=['DESCR_RELACIONAMENTO'])

In [62]:
dfquar20rpvd['DESCR_RELACIONAMENTO'] = dfquar20rpvd['DESCR_RELACIONAMENTO'].map(str.strip)

Por conter um valor NaN na coluna "DESCR_RELACIONAMENTO", eu não conseguiria remover os espaços. Então eu excluí as linhas com esse valor e depois os removi.

In [63]:
dfquar20rpvd = dfquar20rpvd.set_index("DESCR_RELACIONAMENTO")

In [64]:
dfquar20rpvd = dfquar20rpvd.drop("CONHECIDO", axis=0)
dfquar20rpvd = dfquar20rpvd.drop("PARENTESCO", axis=0)
dfquar20rpvd = dfquar20rpvd.drop("NENHUMA RELACAO", axis=0)

In [65]:
dfquar20rpvd.shape

(218, 9)

In [66]:
dfquar19rpvd = dfquar19rpvd.reset_index()
dfquar20rpvd = dfquar20rpvd.reset_index()

Voltei para o índice padrão nos dois dataframes.

In [67]:
dfquar20rpvd['RUBRICA'].value_counts()

Ameaça (art. 147)                104
Injúria (art. 140)                58
Lesão corporal (art. 129)         37
Lesão corporal (art 129 § 9º)      5
Difamação (art. 139)               5
Dano (art. 163)                    5
Calúnia (art. 138)                 3
Art. 213 - Estupro                 1
Name: RUBRICA, dtype: int64

In [68]:
contaRubrica = dfquar20rpvd['RUBRICA'].value_counts()

In [69]:
dfContaRubrica = pd.DataFrame(contaRubrica)

In [70]:
dfContaRubrica = dfContaRubrica.reset_index()

In [71]:
dfContaRubrica.columns = ['Crime', 'Quantidade']

In [72]:
graficoRubrica = px.pie(dfContaRubrica, values='Quantidade', names='Crime', title='Distribuição dos crimes contra mulheres')

In [73]:
graficoRubrica.write_html("rubrica.html")

Criando os gráficos usando a biblioteca Plotly. Primeiro, eu conto os valores da coluna 'RUBRICA' e salvo na variavel contaRubrica. Depois, crio um novo dataframe com os dados da variavél e retorno o index ao valor normal. Assim, consigo renomear as colunas e, por fim, monto o gŕafico e exporto em HTML.

In [74]:
contaProfissao = dfquar20rpvd['DESCR_PROFISSAO'].value_counts()

In [75]:
dfContaProfissao = pd.DataFrame(contaProfissao)

In [76]:
dfContaProfissao = dfContaProfissao.reset_index()

In [77]:
dfContaProfissao.columns = ['Profissão', 'Quantidade']

In [78]:
graficoProfissao = px.bar(dfContaProfissao, y='Profissão', x='Quantidade', title='Profissão')

In [79]:
graficoProfissao.write_html("profissao.html")

In [80]:
contaEscolaridade = dfquar20rpvd['DESCR_GRAU_INSTRUCAO'].value_counts()

In [81]:
dfContaEscolaridade = pd.DataFrame(contaEscolaridade)

In [82]:
dfContaEscolaridade = dfContaEscolaridade.reset_index()

In [83]:
dfContaEscolaridade.columns = ['Escolaridade', 'Quantidade']

In [84]:
graficoEscolaridade = px.bar(dfContaEscolaridade, y='Escolaridade', x='Quantidade', title='Escolaridade')

In [85]:
graficoEscolaridade.write_html("escolaridade.html")